# Read Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('input/data.csv')

# Score

In [ ]:
# Skor tablosu hesaplanıyor >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
score_table=data.drop(['name'],axis=1).groupby('time').apply(lambda x : x.rank()).drop('time',axis=1)
score_table['borda']  =score_table.drop('democracy',axis=1).sum(axis=1)
score_table=score_table.add_suffix('_score')
score_table = pd.concat([data.loc[:,('name','time')],score_table],axis=1)
score_table = score_table.sort_values(['time','democracy_score'],ascending=False)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Rank

In [ ]:
# Her bir yılın, her bir kategorisi için rank hesaplanıyor >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
ranked = score_table.groupby('time').apply(lambda x: x.rank(method='min',ascending=False)).drop(['name','time'],axis=1)
ranked.columns=ranked.columns.str.split('_').str[0]
ranked = ranked.add_suffix('_rank')
ranked =pd.concat([score_table.loc[:,('name','time')],ranked],axis=1)
ranked['difference(democracy-borda)'] = (ranked['democracy_rank'] - ranked['borda_rank']).abs()
ranked = ranked.sort_values(['time','democracy_rank','borda_rank'],ascending=[False,True,True])
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Merge Score & Rank

In [ ]:
# score table ile rank table birleştiriliyor >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
joined = score_table.merge(ranked, on=['name','time'])
joined=joined.sort_values(['time','democracy_rank','borda_rank'],ascending=[False,True,True])
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [ ]:
# sütunlar sıralanıyor >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
column_order =['time', 'name', 'democracy_rank', 'democracy_score', 
       'civil liberties_rank', 'civil liberties_score', 
       'electoral pluralism_rank',
       'electoral pluralism_score', 'government_rank', 'government_score',
       'political culture_rank', 'political culture_score',
       'difference(democracy-borda)','borda_rank', 'borda_score']
joined=joined[column_order]
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Classification

In [ ]:
def defineRange(min_:int,max_:int) -> tuple:
    score = pd.Series(range(min_,max_+1))
    
    maximum:  float = max_
    full_1:   float = score.quantile(.9)
    full_2:   float = score.quantile(.8)
    flawed_1: float = score.quantile(.7)
    flawed_2: float = score.quantile(.6)
    hybrid_1: float = score.quantile(.5)
    hybrid_2: float = score.quantile(.4)
    authoritarian_1: float = score.quantile(.3)
    authoritarian_2: float = score.quantile(.2)
    authoritarian_3: float = min_
    bins_: tuple = (authoritarian_3,authoritarian_2,authoritarian_1,hybrid_2,hybrid_1,flawed_2,flawed_1,full_2,full_1,maximum)
    
    return bins_

In [ ]:
labes_: tuple = ('authoritarian_3','authoritarian_2','authoritarian_1','hybrid_2','hybrid_1','flawed_2','flawed_1','full_2','full_1')


In [ ]:
classified_score = joined.loc[:, ('time','name','borda_score')]

classified_score =classified_score.merge(data.loc[:,('name','time','democracy')], on=['name','time'])

classified_score['category_borda'] = classified_score.groupby('time')['borda_score'].apply(lambda x: pd.cut(x, bins=defineRange(5,5*len(x.dropna())),labels=labes_,include_lowest=True))

classified_score['category_democracy'] = classified_score.groupby('time')['democracy'].apply(lambda x: pd.cut(x, bins=defineRange(1, len(x.dropna()),labels=labes_,include_lowest=True))

# Year by Year

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
j=ranked.loc[:,('time','name','democracy_rank','borda_rank','difference(democracy-borda)')]
date_list = j.time.unique()
df_list = []
for year in date_list:
    df_list.append(j.groupby('time').get_group(year)[['name','democracy_rank','borda_rank','difference(democracy-borda)']].reset_index().sort_values(['democracy_rank','borda_rank']))
year_by_year = pd.concat(df_list,axis=1,keys=date_list)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Write to Excel

In [ ]:
classified_score.to_excel('output/classified_score.xlsx', index=False)
joined.to_excel('output/joined.xlsx',index=False)
year_by_year.to_excel('output/year_by_year.xlsx')

---

In [ ]:
# Ülkelerin, belirli yıldaki skorları arasındaki büyüklükler hesaplanıyor.>>>>>>
st=score_table.drop(['name','time','borda_score','democracy_score'],axis=1)
ranked_row = st.rank(axis=1,ascending=False)
ranked_row = pd.concat([score_table.loc[:,('name','time')],ranked_row],axis=1)
ranked_row.columns=ranked_row.columns.str.split('_').str[0]
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [ ]:
# rank index hesaplanıyor (yıllara göre kategorilerin sıralanması) >>>>>>>>>>>>>>
borda_rank = ranked_row.groupby('time').sum().transpose()
rank_index=borda_rank.copy()
for year in borda_rank.columns:
    rank_index[year] =borda_rank[year].sort_values(ascending=False).index
rank_index = rank_index.reset_index(drop=True)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# Excel'e Yazdır

In [ ]:
borda_rank.to_excel('output/borda_rank.xlsx')
rank_index.to_excel('output/rank_index.xlsx')


---

# Trash